### sys.path 보정

In [ ]:
import os, sys
import keras
PROJECT_ROOT = os.path.abspath("..") 
print("PROJECT_ROOT:", PROJECT_ROOT)
print("Exists src?:", os.path.isdir(os.path.join(PROJECT_ROOT, "src")))
print("List root:", os.listdir(PROJECT_ROOT)[:20])

if PROJECT_ROOT not in sys.path:
    sys.path.insert(0, PROJECT_ROOT)

%load_ext autoreload
%autoreload 2


### Imports & Config 로드

In [ ]:
from src.utils.config import load_config

cfg = load_config("../configs/exp001.yaml")
cfg

### Seed 고정 + run_dir 생성

In [ ]:
import platform
import tensorflow as tf
from src.utils.seed import set_global_seed
from src.utils.io import make_run_dir, copy_config, save_metadata, try_git_commit_hash

set_global_seed(cfg.run.seed)
run_dir = make_run_dir(cfg.run.out_dir, cfg.run.name)
copy_config("../configs/exp001.yaml", run_dir)

save_metadata(
    run_dir,
    {
        "run_dir": run_dir,
        "git_commit": try_git_commit_hash(),
        "python": platform.python_version(),
        "tensorflow": tf.__version__,
        "seed": cfg.run.seed,
        "monitor": cfg.train.monitor,
    },
)
run_dir

### Dataset 생성 (pipeline 재사용)

In [ ]:
from src.data.pipeline import build_datasets

train_ds, val_ds, test_ds, num_classes = build_datasets(cfg)
display("number of classes:", num_classes)
display("train_ds:", train_ds)
display("val_ds:", val_ds)
display("test_ds:", test_ds)

In [ ]:
import matplotlib.pyplot as plt

x, y = next(iter(train_ds))  # type: ignore
plt.figure()
plt.imshow(x[0].numpy())
plt.title(f"label={int(y[0])}")
plt.axis("off")
plt.show()

### Model 생성 + compile

In [ ]:
from src.models.model_factory import build_model

model = build_model(cfg, num_classes=num_classes)
model.summary()

In [ ]:
from src.models.model_factory import build_optimizer

model.compile(
    optimizer=build_optimizer(cfg.train.optimizer, cfg.train.learning_rate),  # type: ignore
    loss=cfg.train.loss,
    metrics=[
        keras.metrics.BinaryAccuracy(name="accuracy"),
        keras.metrics.AUC(name="auc"),
    ],
)

### Callbacks 설정 + fit

In [ ]:
ckpt_dir = os.path.join(run_dir, "checkpoints")
tb_dir = os.path.join(run_dir, "logs")
best_sm = os.path.join(run_dir, "best_savedmodel", "best_savedmodel.keras")

callbacks = [
    keras.callbacks.TensorBoard(log_dir=tb_dir),
    keras.callbacks.ModelCheckpoint(
        filepath=os.path.join(ckpt_dir, "last.weights.h5"),
        save_weights_only=True,
        save_best_only=False,
    ),
    keras.callbacks.ModelCheckpoint(
        filepath=os.path.join(ckpt_dir, "best.weights.h5"),
        save_weights_only=True,
        monitor=cfg.train.monitor,
        mode=cfg.train.monitor_mode,
        save_best_only=True,
    ),
    keras.callbacks.EarlyStopping(
        monitor=cfg.train.monitor,
        mode=cfg.train.monitor_mode,
        patience=cfg.train.early_stop_patience,
        restore_best_weights=True,
    ),
]

In [ ]:
from src.utils.report import export_model_graph


x0, y0 = next(iter(train_ds))  # type: ignore # 배치 1개
export_model_graph(tb_dir, model, (x0, y0), step=1)  # step은 0 말고 1 이상 추천

In [ ]:
history = model.fit(
    train_ds, validation_data=val_ds, epochs=cfg.train.epochs, callbacks=callbacks
)
keras.saving.save_model(model, best_sm)
best_sm

### tensorboard 열기

In [ ]:
from src.utils.report import start_tensorboard, open_in_windows_browser
from src.utils.process import check_port_in_use, kill_process_on_port

# config.report.load_tb_after_fit가 True일 때만 실행
if not cfg.report.load_tb_after_fit:
    exit()

# 포트는 cfg에서 가져옴
config_report_port = cfg.report.tensorboard_port

# model.fit() 끝난 뒤:
url = start_tensorboard(tb_dir, port=config_report_port)  # tb_dir = run_dir/logs
print("TensorBoard:", url)

# VSCode가 포트 포워딩 해주면 Windows에서는 보통 localhost로 접근
open_in_windows_browser(url)

### 학습 시각화

In [ ]:
import pandas as pd

hist = pd.DataFrame(history.history)
hist.tail()

In [ ]:
import matplotlib.pyplot as plt

plt.figure()
plt.plot(hist["loss"], label="loss")
plt.plot(hist["val_loss"], label="val_loss")
plt.legend()
plt.show()

### Eval / Confusion matrix

In [ ]:
import numpy as np
from src.utils.report import save_eval_report

ds = test_ds if test_ds is not None else val_ds

y_true, y_pred = [], []
for bx, by in ds:  # type: ignore
    p = model.predict(bx, verbose=0)  # type: ignore
    pred = (p > 0.5).astype(int)
    y_pred.append(pred.flatten())
    y_true.append(by.numpy())

y_true = np.concatenate(y_true)
y_pred = np.concatenate(y_pred)

In [ ]:
df_head5 = pd.DataFrame({"y_true": y_true, "y_pred": y_pred}).head(20)
display(df_head5)

save_eval_report(run_dir, y_true, y_pred, num_classes)